In [22]:
import pandas as pd
import requests 
import os
import xml.etree.ElementTree as ET
import gzip
import shutil
import urllib



In [37]:

url = "http://data.dot.state.mn.us/iris_xml/incident.xml.gz"
content = urllib.request.urlopen(url)
output = open("incidents.XML.gz", "wb")
output.write(content.read())
output.close()


<Response [200]>

In [2]:
def unzip(file, target):
    with gzip.open(file, 'rb') as f_in:
        with open(target, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)



In [6]:
def parse(XMLfile):
    incident_dirs = []
    roads = []
    locations = []
    names = []
    events = []
    parsedXML = ET.parse(XMLfile)
    root = parsedXML.getroot()
    for child in root:
        try:
            names.append(child.attrib['name'])
        except KeyError:
            name.append("none")
        try:
            incident_dirs.append(child.attrib['dir'])
        except KeyError:
            incident_dir.append("none")
        try:
            roads.append(child.attrib['road'])
        except KeyError:
            roads.append('None')
        try:
            locations.append(child.attrib['location'])
        except KeyError:
            locations.append("none")
        try: 
            events.append(child.attrib['event_type'])
        except KeyError:
            events.append("none")
   

    DF = pd.DataFrame({"name" : names,
                       "direction" :incident_dirs,
                       "Road" : roads,
                       "Location" : locations,
                       "Event" : events
    
                       
    })
    
    print(DF)






In [38]:
unzip("incidents.xml.gz", "incident.xml")

In [39]:
parse("incident.xml")

               name direction     Road               Location  \
0  2018092819553371        WB   T.H.62              @ T.H.100   
1  2018092819531862        SB  U.S.169            @ T.H.62 WB   
2  2018092819533003        NB  U.S.169            @ T.H.62 WB   
3  2018092819315453        NB  T.H.100            @ T.H.62 WB   
4  2018092820101610        SB    I-35W        @ Cleveland Ave   
5      L004_9940258        NB   T.H.77            @ Lindau Ln   
6      L004_9940241        NB  U.S.169  @ Anderson Lakes Pkwy   
7  2018092820040732        SB    I-35E             S of I-35W   
8  2018092820090159        WB   T.H.36        @ Cleveland Ave   
9  2018092819042630        SB  T.H.100            @ T.H.62 WB   

               Event  
0  INCIDENT_ROADWORK  
1  INCIDENT_ROADWORK  
2  INCIDENT_ROADWORK  
3  INCIDENT_ROADWORK  
4  INCIDENT_ROADWORK  
5     INCIDENT_CRASH  
6     INCIDENT_STALL  
7  INCIDENT_ROADWORK  
8  INCIDENT_ROADWORK  
9  INCIDENT_ROADWORK  
